In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

/home/giorgio/virtualenvs/shazura/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
import sys
import os

In [3]:
filenamepaths = glob.glob("./data/validationset/*.png")
filenamepaths[:10]

['./data/validationset/cricket_09879.png',
 './data/validationset/cricket_09553.png',
 './data/validationset/bee_09121.png',
 './data/validationset/cricket_09810.png',
 './data/validationset/cricket_09656.png',
 './data/validationset/noise_09954.png',
 './data/validationset/cricket_09671.png',
 './data/validationset/bee_09354.png',
 './data/validationset/bee_09277.png',
 './data/validationset/cricket_09903.png']

In [4]:
name_to_numeric = {"bee" : 0, "cricket" : 1, "noise" : 2}

def get_class_from_path(path):
    class_name = (os.path.split(path)[-1]).split('_')[0]
    numerical_class = name_to_numeric[class_name]
    return numerical_class

In [5]:
labels = list(map(get_class_from_path, filenamepaths))

In [6]:

for pathname, classname in zip (filenamepaths[:10], labels[:10]):
    print("{} {}".format(classname, pathname))
    

1 ./data/validationset/cricket_09879.png
1 ./data/validationset/cricket_09553.png
0 ./data/validationset/bee_09121.png
1 ./data/validationset/cricket_09810.png
1 ./data/validationset/cricket_09656.png
2 ./data/validationset/noise_09954.png
1 ./data/validationset/cricket_09671.png
0 ./data/validationset/bee_09354.png
0 ./data/validationset/bee_09277.png
1 ./data/validationset/cricket_09903.png


In [7]:
def parse_image_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_png(image_string, channels=1)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize is not actually needed but it is good to have the image dimensions in the
    # Tensor shape
    image = tf.image.resize_images(image, [100, 100])
    return image, label

In [8]:
BATCH_SIZE=100

In [9]:
reset_graph()

The dataset is formed by tuples where the first element is the encoded image and the second is the class

In [10]:
def get_train_inputs(filenames, labels, batch_size, parallel_tasks=4):
    num_samples = len(filenames)
    assert len(filenames) == len(labels), "Filenames and labels should have same length"
    dataset = tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
    dataset = dataset.map(parse_image_function, num_parallel_calls=4)
#     dataset = dataset.shuffle()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    
    iterator = dataset.make_initializable_iterator()
    images, labels = iterator.get_next()
    iterator_init_op = iterator.initializer
    
    inputs = {'images': images, 'labels': labels, 'iterator_init_op': iterator_init_op}
    return inputs
    
train_inputs = get_train_inputs(filenamepaths, labels, BATCH_SIZE)
train_inputs

{'images': <tf.Tensor 'IteratorGetNext:0' shape=(?, 100, 100, 1) dtype=float32>,
 'iterator_init_op': <tf.Operation 'MakeIterator' type=MakeIterator>,
 'labels': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>}

Run a simple iteration to see if the labels are actually changing

In [11]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    sess.run(train_inputs['iterator_init_op'])
    print(sess.run(train_inputs['labels']))
    print(sess.run(train_inputs['labels']))
    print(sess.run(train_inputs['labels']))

[1 1 0 1 1 2 1 0 0 1 0 1 0 1 0 0 2 0 2 2 1 2 2 1 1 2 0 2 0 1 1 1 0 2 1 2 2
 2 0 1 2 2 2 1 1 0 0 0 0 1 1 1 0 0 2 2 2 1 1 1 2 2 2 0 1 0 0 1 1 1 2 0 1 1
 1 2 1 1 2 2 0 2 1 0 2 0 0 0 1 1 0 0 1 0 0 2 1 2 1 1]
[0 2 1 0 2 2 2 1 1 0 1 2 1 0 2 1 2 0 2 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 2
 1 1 1 0 2 0 1 1 0 1 1 1 1 2 1 2 1 0 0 0 0 2 1 1 1 2 2 2 2 0 1 0 0 0 1 0 1
 0 1 2 2 2 0 1 2 2 1 0 1 0 2 0 2 2 1 1 2 1 2 1 2 1 0]
[2 1 1 2 0 2 0 0 1 1 1 0 0 1 0 2 1 2 1 2 1 1 1 1 1 2 1 2 0 1 1 0 2 2 0 0 1
 1 0 0 0 2 2 1 2 1 2 2 2 0 1 1 1 1 0 1 1 2 0 1 1 2 1 0 1 1 1 2 0 1 2 1 1 0
 1 1 2 1 0 2 1 1 0 1 1 2 2 2 1 1 1 2 0 0 1 2 1 2 0 2]


In [12]:
# a simple model, just a perceptron
images = train_inputs["images"]

flatten_images = tf.reshape(images, [-1, 100*100])

dense1 = tf.layers.dense(
    flatten_images,
    100,
    activation=tf.nn.relu,
    kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
    bias_initializer=tf.initializers.random_uniform(0.09, 0.11)
)

dense2 = tf.layers.dense(
    dense1,
    3,
    activation=tf.nn.relu,
    kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
    bias_initializer=tf.initializers.random_uniform(0.09, 0.11)
)


logits=dense2
logits.shape


TensorShape([Dimension(None), Dimension(3)])

In [13]:
labels = train_inputs['labels']
labels = tf.cast(labels, tf.int64)
predictions = tf.argmax(logits,1)

loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
optimizer = tf.train.GradientDescentOptimizer(0.005)
train_op = optimizer.minimize(loss)

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    # actual training
    for epoch in range(3):
        print("Epoch {}".format(epoch))
        # restart from beginning of the dataset
        sess.run(train_inputs['iterator_init_op'])
        for _ in range(len(filenamepaths)//BATCH_SIZE):
            train_op_res, loss_val = sess.run([train_op, loss])
            print(train_op_res)
            print(loss_val)


Epoch 0
None
1.893394
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
Epoch 1
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
Epoch 2
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
None
1.0986122
